In [11]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pickle
import visualkeras

from skimage import color
from skimage import io
from keras.utils import plot_model
from keras import regularizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adamax
from keras.layers import BatchNormalization
from keras import initializers

In [26]:
%load_ext autotime
RANDOM_STATE = 42

In [27]:
MEMORY_LIMIT = 4800
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

time: 973 µs


In [28]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

time: 493 µs


In [29]:
tf.test.is_gpu_available()

True

time: 6.84 ms


In [30]:
tf.test.is_built_with_cuda()

True

time: 2.82 ms


In [31]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

time: 1.26 s


In [32]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.2, shuffle= True)

time: 754 ms


In [42]:
dense_layers = [1,2,3,4]
conv_layers = [2,3,4,5,6,7,8]
opt_layer_1=2
opt_layer_2=1
temp=0
test_acc=0
combinations=0
flag=0

for dense_layer in dense_layers:
    for conv_layer in conv_layers:
        
        print(conv_layer,dense_layer)

        model = Sequential()

        model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3),padding='same',bias_regularizer=regularizers.l2(0.0001),kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.0001)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        for l in range(conv_layer-1):
            model.add(Conv2D(64, (3, 3),padding='same',kernel_initializer='he_normal',bias_regularizer=regularizers.l2(0.0001),kernel_regularizer=regularizers.l2(0.0001)))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            try:
                model.add(MaxPooling2D(pool_size=(2, 2)))
                model.add(Dropout(0.2))
            except:
                model.add(Dropout(0.2))

        
        model.add(Flatten())

        for _ in range(dense_layer):
            model.add(Dense(512))
            model.add(Activation('relu'))

            
        model.add(Dropout(0.5))
        model.add(Dense(10))
        model.add(Activation('softmax'))


        my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
        opt = Adamax(lr = 0.002)
        model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      optimizer=opt,
                      metrics=['accuracy'],
                      )

        model.fit(x_train,y_train,
                  batch_size=64,
                  epochs=50,
                  validation_data=(x_valid,y_valid))
                  #callbacks=[my_callbacks])

        _,test_acc = model.evaluate(x_valid, y_valid)

        print("Current Acc: ",test_acc)
        print("Best Acc: ",temp)
        combinations=combinations+1

        if flag==0 :
            if test_acc > temp :
                opt_layer_1 = conv_layer
                temp=test_acc
            conv_layers = [opt_layer_1]

        elif flag==1 :
            if test_acc > temp :
                opt_layer_2 = dense_layer
                temp=test_acc 
            dense_layers = [opt_layer_2]
    flag=1
    test_acc=0

##################################################################################
print("Trials : ",combinations)
print("Optimal Validation Accuracy: ",temp)
print("The Final Optimal Values for all block is : ")
print('conv_layers: ',opt_layer_1,'\n dense_layers: ',opt_layer_2)

2 1
Epoch 1/50
625/625 [==============================] - 5s 7ms/step - loss: 2.1965 - accuracy: 0.3654 - val_loss: 1.2647 - val_accuracy: 0.5614
Epoch 2/50
625/625 [==============================] - 4s 7ms/step - loss: 1.3033 - accuracy: 0.5483 - val_loss: 1.0848 - val_accuracy: 0.6280
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 1.1147 - accuracy: 0.6134 - val_loss: 0.9859 - val_accuracy: 0.6602
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 0.9687 - accuracy: 0.6691 - val_loss: 1.0057 - val_accuracy: 0.6607
Epoch 5/50
625/625 [==============================] - 4s 7ms/step - loss: 0.8499 - accuracy: 0.7056 - val_loss: 0.8982 - val_accuracy: 0.6963
Epoch 6/50
625/625 [==============================] - 4s 7ms/step - loss: 0.7699 - accuracy: 0.7353 - val_loss: 0.8929 - val_accuracy: 0.7012
Epoch 7/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6897 - accuracy: 0.7588 - val_loss: 0.8392 - val_accuracy: 0.7214
Ep

625/625 [==============================] - 4s 7ms/step - loss: 0.8511 - accuracy: 0.7121 - val_loss: 0.8384 - val_accuracy: 0.7227
Epoch 8/50
625/625 [==============================] - 4s 7ms/step - loss: 0.8012 - accuracy: 0.7297 - val_loss: 0.8173 - val_accuracy: 0.7261
Epoch 9/50
625/625 [==============================] - 4s 7ms/step - loss: 0.7627 - accuracy: 0.7436 - val_loss: 0.8480 - val_accuracy: 0.7280
Epoch 10/50
625/625 [==============================] - 4s 7ms/step - loss: 0.7161 - accuracy: 0.7603 - val_loss: 0.7998 - val_accuracy: 0.7385
Epoch 11/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6800 - accuracy: 0.7735 - val_loss: 0.7661 - val_accuracy: 0.7515
Epoch 12/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6554 - accuracy: 0.7858 - val_loss: 0.8243 - val_accuracy: 0.7448
Epoch 13/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6324 - accuracy: 0.7898 - val_loss: 0.7325 - val_accuracy: 0.7648
Epoch 14/50
6

625/625 [==============================] - 4s 7ms/step - loss: 0.7333 - accuracy: 0.7631 - val_loss: 0.9450 - val_accuracy: 0.7051
Epoch 14/50
625/625 [==============================] - 4s 7ms/step - loss: 0.7168 - accuracy: 0.7703 - val_loss: 0.8235 - val_accuracy: 0.7413
Epoch 15/50
625/625 [==============================] - 4s 7ms/step - loss: 0.7077 - accuracy: 0.7729 - val_loss: 0.8396 - val_accuracy: 0.7339
Epoch 16/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6998 - accuracy: 0.7738 - val_loss: 0.7935 - val_accuracy: 0.7574
Epoch 17/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6708 - accuracy: 0.7869 - val_loss: 0.7703 - val_accuracy: 0.7552
Epoch 18/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6592 - accuracy: 0.7890 - val_loss: 0.7472 - val_accuracy: 0.7704
Epoch 19/50
625/625 [==============================] - 4s 7ms/step - loss: 0.6381 - accuracy: 0.7964 - val_loss: 0.7660 - val_accuracy: 0.7584
Epoch 20/50

625/625 [==============================] - 5s 8ms/step - loss: 0.6929 - accuracy: 0.7887 - val_loss: 0.7879 - val_accuracy: 0.7605
Epoch 20/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6765 - accuracy: 0.7945 - val_loss: 0.7893 - val_accuracy: 0.7564
Epoch 21/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6757 - accuracy: 0.7957 - val_loss: 0.7916 - val_accuracy: 0.7616
Epoch 22/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6627 - accuracy: 0.7989 - val_loss: 0.9117 - val_accuracy: 0.7230
Epoch 23/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6656 - accuracy: 0.7984 - val_loss: 0.7767 - val_accuracy: 0.7654
Epoch 24/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6376 - accuracy: 0.8072 - val_loss: 0.7843 - val_accuracy: 0.7649
Epoch 25/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6428 - accuracy: 0.8063 - val_loss: 0.8022 - val_accuracy: 0.7577
Epoch 26/50

625/625 [==============================] - 5s 9ms/step - loss: 0.6683 - accuracy: 0.8007 - val_loss: 0.7382 - val_accuracy: 0.7788
Epoch 26/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6625 - accuracy: 0.8019 - val_loss: 0.8176 - val_accuracy: 0.7549
Epoch 27/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6508 - accuracy: 0.8093 - val_loss: 0.7177 - val_accuracy: 0.7869
Epoch 28/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6417 - accuracy: 0.8118 - val_loss: 0.7486 - val_accuracy: 0.7790
Epoch 29/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6532 - accuracy: 0.8106 - val_loss: 0.7261 - val_accuracy: 0.7870
Epoch 30/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6363 - accuracy: 0.8153 - val_loss: 0.8449 - val_accuracy: 0.7565
Epoch 31/50
625/625 [==============================] - 5s 8ms/step - loss: 0.6395 - accuracy: 0.8126 - val_loss: 0.7460 - val_accuracy: 0.7745
Epoch 32/50

625/625 [==============================] - 6s 9ms/step - loss: 0.6616 - accuracy: 0.8101 - val_loss: 0.7831 - val_accuracy: 0.7682
Epoch 32/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6502 - accuracy: 0.8148 - val_loss: 0.7502 - val_accuracy: 0.7805
Epoch 33/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6386 - accuracy: 0.8173 - val_loss: 0.7355 - val_accuracy: 0.7840
Epoch 34/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6367 - accuracy: 0.8197 - val_loss: 0.7491 - val_accuracy: 0.7798
Epoch 35/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6346 - accuracy: 0.8189 - val_loss: 0.8102 - val_accuracy: 0.7671
Epoch 36/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6355 - accuracy: 0.8194 - val_loss: 0.7385 - val_accuracy: 0.7864
Epoch 37/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6347 - accuracy: 0.8182 - val_loss: 0.7168 - val_accuracy: 0.7907
Epoch 38/50

625/625 [==============================] - 6s 10ms/step - loss: 0.6699 - accuracy: 0.8116 - val_loss: 0.8303 - val_accuracy: 0.7525
Epoch 38/50
625/625 [==============================] - 6s 10ms/step - loss: 0.6709 - accuracy: 0.8098 - val_loss: 0.8257 - val_accuracy: 0.7581
Epoch 39/50
625/625 [==============================] - 6s 10ms/step - loss: 0.6626 - accuracy: 0.8168 - val_loss: 0.8269 - val_accuracy: 0.7605
Epoch 40/50
625/625 [==============================] - 6s 10ms/step - loss: 0.6716 - accuracy: 0.8143 - val_loss: 0.7234 - val_accuracy: 0.7902
Epoch 41/50
625/625 [==============================] - 6s 10ms/step - loss: 0.6538 - accuracy: 0.8153 - val_loss: 0.7488 - val_accuracy: 0.7825
Epoch 42/50
625/625 [==============================] - 6s 10ms/step - loss: 0.6466 - accuracy: 0.8214 - val_loss: 0.7981 - val_accuracy: 0.7673
Epoch 43/50
625/625 [==============================] - 6s 9ms/step - loss: 0.6537 - accuracy: 0.8182 - val_loss: 0.8183 - val_accuracy: 0.7649
Epoch

625/625 [==============================] - 4s 7ms/step - loss: 0.1332 - accuracy: 0.9699 - val_loss: 1.3029 - val_accuracy: 0.7748
Epoch 44/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1297 - accuracy: 0.9702 - val_loss: 1.1974 - val_accuracy: 0.7891
Epoch 45/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1274 - accuracy: 0.9715 - val_loss: 1.2052 - val_accuracy: 0.7809
Epoch 46/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1295 - accuracy: 0.9725 - val_loss: 1.1768 - val_accuracy: 0.7847
Epoch 47/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1293 - accuracy: 0.9712 - val_loss: 1.1852 - val_accuracy: 0.7875
Epoch 48/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1240 - accuracy: 0.9740 - val_loss: 1.2531 - val_accuracy: 0.7700
Epoch 49/50
625/625 [==============================] - 4s 7ms/step - loss: 0.1180 - accuracy: 0.9759 - val_loss: 1.2958 - val_accuracy: 0.7671
Epoch 50/50

625/625 [==============================] - 4s 7ms/step - loss: 0.0992 - accuracy: 0.9817 - val_loss: 1.4101 - val_accuracy: 0.7660
Epoch 50/50
313/313 [==============================] - 1s 3ms/step - loss: 1.4086 - accuracy: 0.7725
Current Acc:  0.7724999785423279
Best Acc:  0.8026999831199646
3 4
Epoch 1/50
625/625 [==============================] - 6s 8ms/step - loss: 1.7544 - accuracy: 0.3753 - val_loss: 1.1972 - val_accuracy: 0.5817
Epoch 2/50
625/625 [==============================] - 5s 7ms/step - loss: 1.1701 - accuracy: 0.5966 - val_loss: 1.0538 - val_accuracy: 0.6477
Epoch 3/50
625/625 [==============================] - 4s 7ms/step - loss: 0.9762 - accuracy: 0.6693 - val_loss: 0.9049 - val_accuracy: 0.6937
Epoch 4/50
625/625 [==============================] - 5s 7ms/step - loss: 0.8539 - accuracy: 0.7165 - val_loss: 0.8636 - val_accuracy: 0.7183
Epoch 5/50
625/625 [==============================] - 5s 7ms/step - loss: 0.7413 - accuracy: 0.7552 - val_loss: 0.8500 - val_accuracy